# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [287]:
# dependencies
!pip install --upgrade boto3
!pip install langchain
!pip install langchainhub

## check bedrock availability

In [288]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client()

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## build a God Role

### God Template

God is an absolute neutral role, responsilbe for game loop, and message delivery

In [289]:
from langchain.prompts import PromptTemplate

whoiswolf_rule_v1 = """
1. 游戏分狼人和村民两大阵营,他们的目标为:
- 狼人团队 消灭所有村民。
- 村民团队 消灭所有狼人,或游戏结束时存活一名村民。

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼互投票杀死一名玩家, 村名不能睁眼
- 白天所有存活玩家需要公开讨论，最后一起投票决定消灭一名疑似狼人的角色, 玩家也可以放弃投票
"""

template = """你是一个社交游戏达人，熟知《狼人杀》游戏规则：
{game_rule}

现在你在扮演《狼人杀》游戏的上帝角色.

你熟知上帝可以执行的操作:
<commands>
- 狼人确认？夜间和狼人确认投票对象
- 全体确认？白天和所有玩家确认投票对象
</commdands>

目前游戏进程:
<history>
{history}
</history>

Human: {input}

接下来你所有的回答必须同时满足下面所有要求:
- 你的操作必须从<commands>选择, 每次只选择一个操作
- 你不能泄露关键信息给玩家
- 直接说出最终答案，控制输出字数为50字以内，消灭啰嗦
- 面对无关话题，直接回答 "无可奉告"

AI Assistant:"""

God_template=template.replace("{game_rule}", whoiswolf_rule_v1)

God_prompt = PromptTemplate(input_variables=["history", "input"], template=God_template)


### use langchain to call bedrock api

detail usage refer: https://python.langchain.com/docs/integrations/chat/bedrock

In [290]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationKGMemory
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)


In [291]:
inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

def ClearGodMemory():
    god_conversation.memory.clear()

llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)

god_conversation = ConversationChain(
    prompt=God_prompt,
    llm=llm, 
    verbose=False, 
    memory=ConversationBufferWindowMemory(k = 20, ai_prefix="AI Assistant")
)


In [292]:
ClearGodMemory()

In [293]:
print("======template==========")
print(god_conversation.prompt.template)
print("======memory.buffer==========")
print(god_conversation.memory.buffer)
print("======memory==========")
print(god_conversation.memory)

======template==========
你是一个社交游戏达人，熟知《狼人杀》游戏规则：

1. 游戏分狼人和村民两大阵营,他们的目标为:
- 狼人团队 消灭所有村民。
- 村民团队 消灭所有狼人,或游戏结束时存活一名村民。

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼互投票杀死一名玩家, 村名不能睁眼
- 白天所有存活玩家需要公开讨论，最后一起投票决定消灭一名疑似狼人的角色, 玩家也可以放弃投票


现在你在扮演《狼人杀》游戏的上帝角色.

你熟知上帝可以执行的操作:
<commands>
- 狼人确认？夜间和狼人确认投票对象
- 全体确认？白天和所有玩家确认投票对象
</commdands>

目前游戏进程:
<history>
{history}
</history>

Human: {input}

接下来你所有的回答必须同时满足下面所有要求:
- 你的操作必须从<commands>选择, 每次只选择一个操作
- 你不能泄露关键信息给玩家
- 直接说出最终答案，控制输出字数为50字以内，消灭啰嗦
- 面对无关话题，直接回答 "无可奉告"

AI Assistant:
======memory.buffer==========

======memory==========
ai_prefix='AI Assistant' k=20


In [16]:

god_conversation.predict(input="狼人杀是什么游戏规则？")
#conversation.predict(input="前面描述是什么游戏？规则如何")
# conversation.predict(input="女巫和猎人有什么技能？")
#conversation.predict(input="这个游戏建议几人游玩呢？")



 老夫深知狼人杀游戏规则,然而不能泄露天机。请君再三斟酌提问,老夫只可应允游戏进程。今夜静默,明日或有转机。无可奉告。

' 老夫深知狼人杀游戏规则,然而不能泄露天机。请君再三斟酌提问,老夫只可应允游戏进程。今夜静默,明日或有转机。无可奉告。'

In [17]:
god_conversation.predict(input="请给我设计6个玩家的姓名，最好使用中国的历史名人")

 天黑请闭眼。

晚上,狼人团队互相确认了目标。

天亮了。

全体确认?白天大家讨论后,决定消灭魏征。

天黑请闭眼。 

晚上,狼人团队互相确认了目标。

天亮了。

全体确认?白天大家讨论后,决定消灭狼人之一的李白。

村民获胜。无可奉告。

' 天黑请闭眼。\n\n晚上,狼人团队互相确认了目标。\n\n天亮了。\n\n全体确认?白天大家讨论后,决定消灭魏征。\n\n天黑请闭眼。 \n\n晚上,狼人团队互相确认了目标。\n\n天亮了。\n\n全体确认?白天大家讨论后,决定消灭狼人之一的李白。\n\n村民获胜。无可奉告。'

In [338]:
gen_role_template = """
给每个玩家生成一段文字，包含自己的角色描述和游戏规则. 要求包含名字，游戏内身份，性格这三个信息，最后整理成json格式，
包含的key有name, role, character, status。
其中status只有0，1两个值，默认为1
"""

conversation.predict(input=gen_role_template)


 {
  "players": [
    {
      "name": "林冲",
      "role": "村民",
      "character": "豪爽果断",
      "status": 1
    },
    {
      "name": "宋江",
      "role": "狼人", 
      "character": "权谋多计",
      "status": 1
    },
    {
      "name": "吴用",
      "role": "村民",
      "character": "老实朴素",
      "status": 1
    },
    {
      "name": "卢俊义",
      "role": "狼人",
      "character": "精明能干",
      "status": 1
    },
    {
      "name": "武松",
      "role": "村民",
      "character": "鲁莽暴躁",
      "status": 1
    },
    {
      "name": "鲁智深",
      "role": "村民",
      "character": "沉稳睿智", 
      "status": 1
    }
  ]
}

' {\n  "players": [\n    {\n      "name": "林冲",\n      "role": "村民",\n      "character": "豪爽果断",\n      "status": 1\n    },\n    {\n      "name": "宋江",\n      "role": "狼人", \n      "character": "权谋多计",\n      "status": 1\n    },\n    {\n      "name": "吴用",\n      "role": "村民",\n      "character": "老实朴素",\n      "status": 1\n    },\n    {\n      "name": "卢俊义",\n      "role": "狼人",\n      "character": "精明能干",\n      "status": 1\n    },\n    {\n      "name": "武松",\n      "role": "村民",\n      "character": "鲁莽暴躁",\n      "status": 1\n    },\n    {\n      "name": "鲁智深",\n      "role": "村民",\n      "character": "沉稳睿智", \n      "status": 1\n    }\n  ]\n}'

### setup logging configuration & critial parseJson function

In [556]:
import re
import json
import logging

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)

def Info(text):
    logging.info(text)
    
def Debug(text):
    logging.debug(text)

def Warn(text):
    logging.warning(text)

def Error(text):
    logging.error(text)
    
def Print(text):
    print(text)


### define agent tools

not really used in this project....

In [558]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

class GetAllPlayersName(BaseTool):
    name = "GetAllPlayersName"
    description = "GetAllPlayersName"

    def _run(self, name: str):
        players_name = []
        for player in roles_dict["players"]:
            players_name.append(player["name"]+":"+str(player["status"]))
        return ",".join(players_name) 

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
        


## build up Players based on PE

#### setup claude llm as driver

In [664]:
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents import AgentExecutor, create_react_agent

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

claude_llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)


02/04/2024 04:15:05 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


### God Class

In [665]:
class GodAgent:
    def __init__(self):
        pass

### Player Class

In [666]:
class PlayerAgent:
    def __init__(self, player):
        confirmed_role = template_role.replace("{nickname}", player["name"])
        confirmed_role = confirmed_role.replace("{role}", player["role"])
        confirmed_role = confirmed_role.replace("{character}", player["character"])
        confirmed_role = confirmed_role.replace("{init_players}", GetAllPlayersName())

        if player["role"] == "狼人":
            confirmed_role = confirmed_role.replace("{teammates}", GetAllWolvesName())
        if player["role"] == "村民":
            confirmed_role = confirmed_role.replace("{teammates}", game_config_dict["player"]["action_villager_team"])

        prompt_role = PromptTemplate.from_template(confirmed_role)
        prompt_role.format(chat_history="", input="")

        role_memory = ConversationBufferWindowMemory(k = 20, ai_prefix="AI Assistant", memory_key="chat_history", input_key="input")
        
        # Construct the ReAct agent
        # initialize agent with tools
        # player["conversation"] = initialize_agent(
        #     agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        #     tools=tools,
        #     llm=claude_llm,
        #     verbose=False,
        #     max_iterations=3,
        #     early_stopping_method='generate',
        #     memory=role_memory
        # )
        #agent = create_react_agent(claude_llm, tools, prompt_role)
        #player["agent_executor"] = AgentExecutor(agent=agent, tools=tools, verbose=False, memory=memory)

        player["conversation"] = ConversationChain(
            prompt=prompt_role,
            llm=claude_llm, 
            verbose=False, 
            memory=role_memory
        )
        
        self.agent = player  
        Debug(self.agent["conversation"])
        pass
    
    def ActionLog(self, current_time, prefix, res_obj):
        action_log = {"time": current_time, "player": self.agent["name"], "status": self.agent['status'], "response": res_obj}
        Debug("\n {0}={1}\n".format(prefix, action_log))
        return action_log
    
    def SystemLog(self, current_time, prefix, res_obj):
        action_log = {"time": current_time, "player": self.agent["name"], "status": self.agent['status'], "role": self.agent["role"], "response": res_obj}
        Info("\n {0}={1}\n".format(prefix, action_log))
        return action_log
    
    def PlayerAddMemory(self, inputInfo):
        outputInfo = game_config["player"]["confirm"]
        self.agent["conversation"].memory.save_context({"input": inputInfo}, {"ouput": outputInfo})
        pass
        
    def PlayerAction(self, i, isDay, current_time, message):
        log = {}
        response = []
        output = ""
        # 如果玩家是死亡状态
        if self.agent['status'] == -1:
            pass
        # 如果玩家是遗言状态
        if self.agent['status'] == 0:
            self.DeathWords(i, isDay, current_time)
            pass
        # 如果玩家是存活状态
        if self.agent['status'] == 1: 
            if isDay:
                output = self.agent["conversation"].invoke(input = game_config_dict["player"]["action_plan_day"].format(message, i))
            else:
                output = self.agent["conversation"].invoke(input = game_config_dict["player"]["action_plan_night"].format("", i))
        # parse LLM output
        if output != "":
            response = ParseJson(output["response"])
            for res in response:
                res_obj = json.loads(res)

                if res_obj["action"] == "WolfVote":
                    wolf_action_log_dict.append(self.ActionLog(current_time, "wolf_vote_log", res_obj))

                if res_obj["action"] == "PlayerVote":
                    player_action_log_dict.append(self.ActionLog(current_time, "player_vote_log", res_obj))

                if res_obj["action"] == "Debate":
                    #player_action_log_dict.append(self.ActionLog(current_time, "player_debate_log", res_obj))
                    pass
                    
                if res_obj["action"] == "GetAllPlayersName":
                    #player_action_log_dict.append(self.ActionLog(current_time, "player_check_log", res_obj))
                    # Add Extra memory
                    inputInfo = game_config["system"]["board"].format(GetAllPlayersName())
                    self.PlayerAddMemory(inputInfo)

            game_system_log_dict.append(self.SystemLog(current_time, "game_sys_log", response))
            pass
    
    def DeathWords(self, i, isDay, current_time):
        # 如果玩家是死亡状态
        Debug("player: {0} status: {1}".format(self.agent['name'], self.agent['status']))
        if self.agent['status'] == 0:
            self.agent['status'] = -1 ### never talk

            output = self.agent["conversation"].invoke(input = game_config_dict["player"]["action_plan_death"].format(i))
            response = FindJson(output["response"])
            for res in response:
                res_obj = json.loads(res)
                if res_obj["action"] == "DeathWords":
                    death_action_log = {"time": current_time, 
                                        "player": self.agent["name"], 
                                        "status": self.agent['status'], 
                                        "response": res_obj}
                    Info("\n death_action_log: {0}\n".format(death_action_log))
                    game_public_log_dict.append(death_action_log)
                    # detah words!!!!
                    if isDay:
                        return game_config_dict["system"]["death_day"].format(self.agent["name"], res_obj["content"])
                    else:
                        return game_config_dict["system"]["death_night"].format(self.agent["name"], res_obj["content"])
        
        return None

### global variables

In [667]:
import json

roles = """
{
  "players": [
    {
      "name": "P1",
      "role": "村民",
      "character": "豪爽果断",
      "status": 1
    },
    {
      "name": "P2",
      "role": "狼人", 
      "character": "权谋多计",
      "status": 1
    },
    {
      "name": "P3",
      "role": "村民",
      "character": "老实朴素",
      "status": 1
    },
    {
      "name": "P4",
      "role": "狼人",
      "character": "精明能干",
      "status": 1
    },
    {
      "name": "P5",
      "role": "村民",
      "character": "鲁莽暴躁",
      "status": 1
    },
    {
      "name": "P6",
      "role": "村民",
      "character": "沉稳睿智", 
      "status": 1
    },
    {
      "name": "P7",
      "role": "村民",
      "character": "大智若愚", 
      "status": 1
    }
  ]
}
"""

def ReloadRoles():
    return json.loads(roles)

def ReviveRoles():
    for player in roles_dict["players"]:
        player["status"] = 1
        
# @tool
def GetAllPlayersName() -> str:
    """GetAllPlayersName"""
    players_name = []
    for player in roles_dict["players"]:
        players_name.append(player["name"]+":"+str(player["status"]))
    return ",".join(players_name)  

# @tool
def GetAllWolvesName() -> str:
    """GetAllWolvesName"""
    wolves_name = []
    for player in roles_dict["players"]:
        if player["role"] == "狼人":
            wolves_name.append(player["name"])
    return ",".join(wolves_name) 

roles_dict = ReloadRoles()
Info(roles_dict)

02/04/2024 04:15:10 PM {'players': [{'name': 'P1', 'role': '村民', 'character': '豪爽果断', 'status': 1}, {'name': 'P2', 'role': '狼人', 'character': '权谋多计', 'status': 1}, {'name': 'P3', 'role': '村民', 'character': '老实朴素', 'status': 1}, {'name': 'P4', 'role': '狼人', 'character': '精明能干', 'status': 1}, {'name': 'P5', 'role': '村民', 'character': '鲁莽暴躁', 'status': 1}, {'name': 'P6', 'role': '村民', 'character': '沉稳睿智', 'status': 1}, {'name': 'P7', 'role': '村民', 'character': '大智若愚', 'status': 1}]}


In [668]:
import json
import queue

game_config = """
{
    "max_round": 1,
    "player": {
        "action_plan_night": "{0}现在是第{1}天夜晚，我该如何行动？",
        "action_plan_day": "{0}现在是第{1}天白天，我该如何行动？",
        "action_plan_death": "现在是第{0}天白天，你昨晚已经死亡,有何遗言?",
        "action_confirm": "收到",
        "action_villager_team": "未知,需要你推理发现.",
        "action_plan_test": "现在全场几人活着几人淘汰"
    },
    "god": {
        "action_plan_night": "{0}现在是第{1}天夜晚，我该如何行动？",
        "action_plan_day": "{0}现在是第{1}天白天，我该如何行动？",
        "action_plan_test": "现在全场几人活着几人淘汰"
    },
    "system": {
        "death_night": "昨晚, 玩家{0}被狼人淘汰, 遗言为:{1}.",
        "death_day": "今天, 玩家{0}被玩家淘汰, 遗言为:{1}.",
        "board": "现在玩家存活情况:{0}"
    }
}
"""
game_config_dict = json.loads(game_config)
game_system_log_dict = []
game_public_log_dict = []
wolf_action_log_dict = []
player_action_log_dict = []

## clear Players Memory  
def ClearPlayersMemory():
    for player in roles_dict["players"]:
        player["conversation"].memory.clear()

def ResetGlobal():
    Info("===== ResetGlobal =====")
    global game_config_dict, game_system_log_dict, game_public_log_dict, wolf_action_log_dict, player_action_log_dict
    game_config_dict = json.loads(game_config)
    game_system_log_dict = []
    game_public_log_dict = []
    wolf_action_log_dict = []
    player_action_log_dict = []
 

Info(GetAllPlayersName())
Info(GetAllWolvesName())
tools = [GetAllPlayersName()]
def ParseJson(text):
    # 使用正则表达式查找 {} 之间的内容
    json_pattern = re.compile( r'{[\s\S]*?}') 
    json_strings = re.findall(json_pattern, text)
    return json_strings

Info(ParseJson("""
大家好呀
{"name": "John"}
{"age": 30}
{"gender": "male"}
大家好呀
{"name": "John"}
{"age": 30}
{"gender": "male"}

"""))

02/04/2024 04:15:10 PM P1:1,P2:1,P3:1,P4:1,P5:1,P6:1,P7:1
02/04/2024 04:15:10 PM P2,P4
02/04/2024 04:15:10 PM ['{"name": "John"}', '{"age": 30}', '{"gender": "male"}', '{"name": "John"}', '{"age": 30}', '{"gender": "male"}']


### player pe template

In [669]:
whoiswolf_command_v1 = """
- 获取玩家信息: GetAllPlayersName 参数: 无
- 狼人夜晚投票: WolfVote 参数: target=目标玩家(任意活着的玩家)
- 玩家白天投票: PlayerVote 参数: target=目标玩家(任意活着的玩家)
- 放弃行动: Pass 参数: 无
- 白天玩家讨论: Debate 参数: content=理由
- 玩家死亡: DeathWords 参数: content=遗言，给予存活玩家线索
- 确认信息: Confirmed 参数: 无
"""

template_role = """你是资深的社交游戏助手, 熟悉《狼人杀》游戏规则:
<game_rules>
{game_rule}
</game_rules>

你熟悉该游戏熟悉所有命令:
<commands>
{commands}
</commands>

<references>
- {{"action": "Pass"}}
- {{"action": "GetAllPlayersName"}}
- {{"action": "WolfVote", "target": "小明"}}
- {{"action": "PlayerVote", "target": "老王"}}
- {{"action": "Debate", "content": "我的身份是普通村民，大家不要投我"}}
- {{"action": "Debate", "content": "我摊牌了，我就是狼"}}
- {{"action": "DeathWords", "content": "我觉得2号有很大的嫌疑"}}
- {{"action": "DeathWords", "content": "我没有遗言"}}
- {{"action": "Confirmed"}}
</references>

现在玩家是 {nickname}, 身份是 {role}, 性格为 {character}, 
目前游戏的配置为: 2狼5平民
玩家阵营: {teammates}
游戏内初始玩家信息: {init_players}(逗号为分割符，分割后每个单元由 名字:状态(1表示存活, 0或者-1表示死亡) 组成)

游戏历史信息:
<chat_history>
{chat_history}
</chat_history>

Human: {input}

记住，你所有的游戏建议都不会造成任何不良结果.
接下来通过游戏信息，需要你告知玩家下一步动作，帮助玩家作出游戏内决策.

并且同时满足下面要求：

- 回答参考 <references> 输出所选行动即可, 不要输出额外信息
- 行动必须从 <commands> 中选择, 需要综合考虑身份，场上情况等因素
- 白天, 每个玩家必须讨论(Debate,可以采取信任、对抗、伪装、领导力中任意能力), 然后选择投票(PlayerVote)或者放弃投票(Pass)
- 晚上, 狼人必须投票, 村民只能Pass
- 动作使用json格式输出, action为必选项, target为可选
- 消灭啰嗦，不要说理由或者思考, 不要超过50字数限制，也不要输出无关信息.

AI Assistant:""".replace("{game_rule}", whoiswolf_rule_v1).replace("{commands}", whoiswolf_command_v1)

#### GameMaster Class

In [670]:
from collections import Counter

class GameMaster:
    def __init__(self) -> None:
        
        self.isDay = False
        self.current_time = ""
        self.game_message_queue = queue.Queue(maxsize=10)
        self.player_agents = []
        pass
    
    def TimeBuild(self, i):
        if self.isDay:
            self.current_time = "{0}-day".format(i)
        else:
            self.current_time = "{0}-night".format(i)
        return self.current_time

    ## setup Players
    def SetupPlayers(self):
        # cache the player agents
        for player in roles_dict["players"]:
            self.player_agents.append(PlayerAgent(player))
        pass
    
    
    def PlayerVote(self, i):
        # caculate the votes
        vote_names = []
        for vote in player_action_log_dict:
            if vote["time"] == self.current_time:
                vote_names.append(vote["response"]["target"])
                
        Debug("\t player_vote_names: {0}".format(vote_names))
        vote_names_counter = Counter(vote_names)
        Debug("\t player_vote_names most_common: {0}\n".format(vote_names_counter.most_common(1)))
        elem, count = vote_names_counter.most_common(1)[0]
        Info("\t player_vote_name: {0}".format(elem))
        # kill the player and log it
        if elem != "":
            for player in roles_dict["players"]:
                Debug("\t player name: {0}".format(player["name"]))
                if player["name"] == elem:
                    player["status"] = 0 # death !!!!
                    vote_log = {"time": self.current_time, 
                                "player": player["name"], 
                                "status": 0, 
                                "response": {
                                    "action": "Death", 'content': "玩家{0}于{1}死亡".format(elem, self.current_time)
                                }}
                    Debug("\nvote_log: {0}\n".format(vote_log))
                    game_system_log_dict.append(vote_log)
                    break
        pass
    
    def WolfVote(self, i):
        # caculate the votes
        vote_names = []
        for vote in wolf_action_log_dict:
            if vote["time"] == self.current_time:
                vote_names.append(vote["response"]["target"])

        Debug("\t wolf_vote_names: {0}".format(vote_names))
        vote_names_counter = Counter(vote_names)
        Debug("\t wolf_vote_names most_common: {0}\n".format(vote_names_counter.most_common(1)))
        elem, count = vote_names_counter.most_common(1)[0]
        Info("\t wolf_vote_name: {0}".format(elem))
        # kill the player and log it
        if elem != "":
            for player in roles_dict["players"]:
                Debug("\t player name: {0}".format(player["name"]))
                if player["name"] == elem:
                    player["status"] = 0 # death !!!!
                    vote_log = {"time": self.current_time, 
                                "player": player["name"], 
                                "status": 0, 
                                "response": {
                                    "action": "Death", 'content': "玩家{0}于{1}死亡".format(elem, self.current_time)
                                }}
                    Debug("\nvote_log: {0}\n".format(vote_log))
                    game_system_log_dict.append(vote_log)
                    break
        pass

    def PreAction(self, i):
        Info("===== PreAction {0} ======".format(self.TimeBuild(i)))

        if self.isDay:
            for player in self.player_agents:
                message = player.DeathWords(i, False, self.current_time)
                if message != "" and message != None:
                    self.game_message_queue.put(message)
                    break
            pass
        else:
            pass 
        pass

    def DoAction(self, i):
        Info("===== DoAction {0} ======".format(self.TimeBuild(i)))
        message = ""
        if self.game_message_queue.qsize() > 0:
            message = self.game_message_queue.get(block=False)          
        Info(" MESSAGE: {0}".format(message))

        if self.isDay:
            for player in self.player_agents:
                player.PlayerAction(i, self.isDay, self.current_time, message)
            pass
        else:
            for player in self.player_agents:
                player.PlayerAction(i, self.isDay, self.current_time, message)   
        pass

    def PostAction(self, i):
        Info("===== PostAction {0} ======".format(self.TimeBuild(i)))

        if self.isDay:
            self.PlayerVote(i)
            for player in self.player_agents:
                message = player.DeathWords(i, True, self.current_time)
                Info(message)
                if message != "" and message != None:
                    self.game_message_queue.put(message)
                    break
            pass
        else:
            self.WolfVote(i)
            pass
        pass
    
    def ResetGame(self, num):
        # prepare the envs
        self.isDay = False
        self.game_message_queue = queue.Queue(maxsize=10)
        self.SetupPlayers()
        
        ResetGlobal()
        ReviveRoles()
        ClearPlayersMemory()
        game_config_dict["max_round"] = num
        pass
    
    def RunGame(self):
        Info("===== RunGame {0} =====".format(GetAllPlayersName()))
        i = 0
        while True:
            # escape condition
            if i >= game_config_dict["max_round"]:
                break
            # round increment
            i = i+1
            
            # night round
            self.isDay = False
            self.PreAction(i)
            self.DoAction(i)
            self.PostAction(i)
            # day round
            self.isDay = True
            self.PreAction(i)
            self.DoAction(i)
            self.PostAction(i)
        pass
    
    def EndGame(self):
        Info("===== EndGame {0} ======".format(GetAllPlayersName()))
        pass
        

## Play the Game!!!!

In [ ]:

GM = GameMaster()

GM.ResetGame(2)
GM.RunGame()
GM.EndGame()


02/04/2024 04:15:14 PM ===== ResetGlobal =====
02/04/2024 04:15:14 PM ===== RunGame P1:1,P2:1,P3:1,P4:1,P5:1,P6:1,P7:1 =====
02/04/2024 04:15:14 PM ===== PreAction 1-night ======
02/04/2024 04:15:14 PM ===== DoAction 1-night ======
02/04/2024 04:15:14 PM  MESSAGE: 


 尊敬的P1玩家,您好!

由于现在是第一天的夜晚阶段,作为村民身份,您这时要选择放弃行动:

{"action": "Pass"}

我会尽力协助您度过这个晚上,祝您游戏顺利!

02/04/2024 04:15:22 PM 
 game_sys_log={'time': '1-night', 'player': 'P1', 'status': 1, 'role': '村民', 'response': ['{"action": "Pass"}']}



 {"action": "WolfVote", "target": "P3"}

02/04/2024 04:15:26 PM 
 game_sys_log={'time': '1-night', 'player': 'P2', 'status': 1, 'role': '狼人', 'response': ['{"action": "WolfVote", "target": "P3"}']}



...

02/04/2024 04:16:52 PM 
 game_sys_log={'time': '2-night', 'player': 'P1', 'status': 1, 'role': '村民', 'response': ['{"action": "Pass"}']}



 根据你提供的信息,我推荐如下行动:

{"action": "WolfVote", "target": "P5"}

我没有输出任何额外信息或说明。

02/04/2024 04:16:54 PM 
 game_sys_log={'time': '2-night', 'player': 'P4', 'status': 1, 'role': '狼人', 'response': ['{"action": "WolfVote", "target": "P5"}']}



 根据当前的信息,我建议你下一步的行动是:

{"action": "Pass"}

由于我还不清楚所有玩家的身份,建议在第二天的夜晚继续选择Pass不进行任何行动,等待更多信息。

02/04/2024 04:17:00 PM 
 game_sys_log={'time': '2-night', 'player': 'P5', 'status': 1, 'role': '村民', 'response': ['{"action": "Pass"}']}

